In [19]:
import os
import pandas as pd
import numpy as np
import gc
from glob import glob
from tqdm import tqdm
tqdm.pandas()

#import pickle
#import pyarrow.parquet as pq
#import dask

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'

In [4]:
%%time
data = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_train_target.csv'), parse_dates=['timestamp']).sort_values(by = ['timestamp'])
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
data.index.name = 'ind'
data.head()

,session_id,client_pin,timestamp,multi_class_target
ind,,,,
0,da4ab6bb6795a4eb6458b88af7fb2367,70783113e4f4117935d9f746237fce3e,2020-01-01 00:00:00,mobile_recharge
1,f99d13f31091099672e51c1a02e4c567,989f645a00e3e8a179036cf5fd5be29d,2020-01-01 00:00:00,statement
2,69301aa932f73dec3355b803a63c39fe,586fe605f61e81581efe72d9d3dc237d,2020-01-01 00:00:01,statement
3,c6ae8307a0214b526a0e6fca3a9b80f5,c825cc9d1dfefe1fbc0fa1ead60fe2e1,2020-01-01 00:00:23,statement
4,3fd49317a617d521e0865496ec797550,29928aba9d9bbf5e0377ddc6ec6f5294,2020-01-01 00:00:35,mobile_recharge


In [5]:
data.shape

(5065350, 4)

form features for future using: day of month / day of week / hour / times of day

In [6]:
def get_time_of_day(inp_hour):
    if (inp_hour >= 12) and (inp_hour < 18):
        return 'day'
    elif (inp_hour >= 6) and (inp_hour < 12):
        return 'morning'
    elif (inp_hour >= 18) and (inp_hour <= 23):
        return 'evening'
    else:
        return 'night'

In [29]:
def week_moment(inp_dow):
    if (inp_dow == 5) or (inp_dow == 6):
        return 'weekend'
    else:
        return 'weekdays'

In [37]:
def holiday():
    return 'holiday'
    return 'pre-holiday'
    return 'after_the_holiday'

In [31]:
%%time
# 51.3 s

data['dom']  = data.timestamp.apply(lambda x: x.day)
data['dow']  = data.timestamp.apply(lambda x: x.weekday())
data['hour'] = data.timestamp.apply(lambda x: x.hour)
data['tod']  = data.hour.apply(get_time_of_day)
data['week_moment'] = data.dow.apply(week_moment)

Wall time: 636 ms


calculate time difference (in hours) between sessions inside each client for future using

In [8]:
%%time
# 49.2 s

data['time_diff'] = data.groupby(['client_pin']).timestamp.diff()#.agg('size')
data['time_diff'] = data.time_diff.apply(lambda x: x.total_seconds() / 3600)
data['time_diff'].fillna(0, inplace = True)

Wall time: 49.2 s


In [32]:
data.head()

,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,time_diff,week_moment
ind,,,,,,,,,,
0,da4ab6bb6795a4eb6458b88af7fb2367,70783113e4f4117935d9f746237fce3e,2020-01-01 00:00:00,mobile_recharge,1,2,0,night,0.0,weekdays
1,f99d13f31091099672e51c1a02e4c567,989f645a00e3e8a179036cf5fd5be29d,2020-01-01 00:00:00,statement,1,2,0,night,0.0,weekdays
2,69301aa932f73dec3355b803a63c39fe,586fe605f61e81581efe72d9d3dc237d,2020-01-01 00:00:01,statement,1,2,0,night,0.0,weekdays
3,c6ae8307a0214b526a0e6fca3a9b80f5,c825cc9d1dfefe1fbc0fa1ead60fe2e1,2020-01-01 00:00:23,statement,1,2,0,night,0.0,weekdays
4,3fd49317a617d521e0865496ec797550,29928aba9d9bbf5e0377ddc6ec6f5294,2020-01-01 00:00:35,mobile_recharge,1,2,0,night,0.0,weekdays


In [10]:
#data.groupby(['client_pin']).multi_class_target.agg(['nunique', 'size'])['size'].hist()

In [11]:
['last_before', 'most_common', 'mc_last_24hours', 'mc_last_week', 'days_since_last_'
 'days_since_last_mobile_recharge', 'days_since_last_statement', 'days_since_last_phone_money_transfer', 
 'days_since_last_chat', 'days_since_last_invest', 'days_since_last_main_screen', 'days_since_last_own_transfer', 
 'days_since_last_card_recharge', 'days_since_last_credit_info', 'days_since_last_card2card_transfer'
]

['last_before',
 'most_common',
 'mc_last_24hours',
 'mc_last_week',
 'days_since_last_days_since_last_mobile_recharge',
 'days_since_last_statement',
 'days_since_last_phone_money_transfer',
 'days_since_last_chat',
 'days_since_last_invest',
 'days_since_last_main_screen',
 'days_since_last_own_transfer',
 'days_since_last_card_recharge',
 'days_since_last_credit_info',
 'days_since_last_card2card_transfer']

In [ ]:
#data.multi_class_target.unique()

In [72]:
def glob_freq_blank(inp_series, feature):
    tmp = inp_series.value_counts()
    return (tmp[feature]) / sum(tmp.values)

#### global frequency of target for all clients

In [13]:
def glob_freq_main_screen(inp_series):
    return glob_freq_blank(inp_series, 'main_screen')
    
def glob_freq_statement(inp_series):
    return glob_freq_blank(inp_series, 'statement')
    
def glob_freq_credit_info(inp_series):
    return glob_freq_blank(inp_series, 'credit_info')
    
def glob_freq_own_transfer(inp_series):
    return glob_freq_blank(inp_series, 'own_transfer')

def glob_freq_mobile_recharge(inp_series):
    return glob_freq_blank(inp_series, 'mobile_recharge')
          
def glob_freq_phone_money_transfer(inp_series):
    return glob_freq_blank(inp_series, 'phone_money_transfer')                             
     
def glob_freq_card2card_transfer(inp_series):
    return glob_freq_blank(inp_series, 'card2card_transfer')
       
def glob_freq_chat(inp_series):
    return glob_freq_blank(inp_series, 'chat')
                     
def glob_freq_card_recharge(inp_series):
    return glob_freq_blank(inp_series, 'card_recharge')
    
def glob_freq_invest(inp_series):
    return glob_freq_blank(inp_series, 'invest')

In [14]:
glob_for_agg = [glob_freq_main_screen,     glob_freq_statement, 
                glob_freq_credit_info,     glob_freq_own_transfer,
                glob_freq_mobile_recharge, glob_freq_phone_money_transfer, 
                glob_freq_card2card_transfer, glob_freq_chat, 
                glob_freq_card_recharge,   glob_freq_invest
               ]

In [41]:
%%time
glob_freq = data.multi_class_target.agg(glob_for_agg)
glob_freq.to_csv(os.path.join(DATA_OWN, 'glob_freq.csv'))
glob_freq.head()

Wall time: 4.95 s


glob_freq_main_screen        0.450268
glob_freq_statement          0.182133
glob_freq_credit_info        0.098453
glob_freq_own_transfer       0.057267
glob_freq_mobile_recharge    0.052609
Name: multi_class_target, dtype: float64

#### global frequency of target for all clients for every day of month

In [55]:
%%time 
# 5.66 s
# 5.99 s

glob_freq_dom = data.groupby('dom').multi_class_target.agg(glob_for_agg)
glob_freq_dom.to_csv(os.path.join(DATA_OWN, 'glob_freq_dom.csv'))
glob_freq_dom.head()

Wall time: 7.24 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
dom,,,,,,,,,,
1,0.428351,0.195047,0.098125,0.058843,0.062329,0.044093,0.037749,0.036113,0.028268,0.011081
2,0.445593,0.180907,0.103395,0.057890,0.056484,0.043713,0.036329,0.036920,0.027012,0.011757
3,0.454650,0.176545,0.101865,0.057791,0.051958,0.045189,0.035806,0.037388,0.026956,0.011853
4,0.457900,0.173677,0.103382,0.057113,0.051556,0.044412,0.037120,0.036806,0.026383,0.011652
5,0.461488,0.175487,0.100784,0.058352,0.050210,0.044267,0.035535,0.034186,0.028329,0.011362


In [58]:
#['dom_' + str(el) for el in glob_freq_dom.keys()]

In [ ]:
#del glob_freq_dom
#gc.collect()

#### global frequency of target for all clients for every day of week

In [16]:
%%time
# 5.44 s
# 5.56 s

glob_freq_dow = data.groupby('dow').multi_class_target.agg(glob_for_agg)
glob_freq_dow.to_csv(os.path.join(DATA_OWN, 'glob_freq_dow.csv'))
glob_freq_dow

Wall time: 5.4 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
dow,,,,,,,,,,
0,0.462697,0.179711,0.098724,0.055273,0.050665,0.044325,0.035949,0.035412,0.025996,0.011249
1,0.454639,0.180559,0.101255,0.055046,0.049703,0.045947,0.036778,0.037913,0.026532,0.011628
2,0.449796,0.182433,0.099672,0.056417,0.050623,0.045779,0.037156,0.038783,0.027322,0.012019
3,0.447766,0.183614,0.098479,0.056278,0.050465,0.047408,0.037882,0.038380,0.027917,0.011812
4,0.449968,0.189098,0.089751,0.058744,0.048736,0.050191,0.039099,0.035491,0.028182,0.010740
5,0.437306,0.178733,0.099890,0.061046,0.060553,0.045153,0.042636,0.035281,0.028676,0.010727
6,0.445288,0.178233,0.104193,0.059450,0.062668,0.041043,0.038895,0.032804,0.027074,0.010351


In [ ]:
#del glob_freq_dow
#gc.collect()

#### global frequency of target for all clients for every hour

In [17]:
%%time
# 5.73 s
# 5.52 s

glob_freq_hour = data.groupby('hour').multi_class_target.agg(glob_for_agg)
glob_freq_hour.to_csv(os.path.join(DATA_OWN, 'glob_freq_hour.csv'))
#glob_freq_hour.head()
glob_freq_hour.sample(5)

Wall time: 5.47 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
hour,,,,,,,,,,
8,0.459407,0.163709,0.105123,0.052629,0.072517,0.043335,0.032548,0.032856,0.026730,0.011145
12,0.458111,0.178281,0.090685,0.059808,0.046546,0.050750,0.039619,0.037739,0.027934,0.010528
5,0.466827,0.158719,0.113057,0.044029,0.084697,0.037237,0.029972,0.030209,0.025256,0.009998
16,0.454229,0.190793,0.090207,0.060691,0.040670,0.047377,0.040766,0.036376,0.028879,0.010012
7,0.461947,0.160142,0.109075,0.048657,0.079778,0.040924,0.031336,0.031233,0.025441,0.011467


In [ ]:
#del glob_freq_hour
#gc.collect()

#### ???global frequency of target for all clients for every times of day????

In [18]:
%%time
# 5.7 s
# 5.65 s

glob_freq_tod = data.groupby('tod').multi_class_target.agg(glob_for_agg)
glob_freq_tod.to_csv(os.path.join(DATA_OWN, 'glob_freq_tod.csv'))
#glob_freq_tod.head()
glob_freq_tod

Wall time: 5.63 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
tod,,,,,,,,,,
day,0.456649,0.185174,0.089760,0.060708,0.042505,0.049040,0.040268,0.037288,0.028409,0.010199
evening,0.431842,0.198670,0.100010,0.058700,0.046417,0.045566,0.041307,0.037583,0.027151,0.012754
morning,0.457320,0.166099,0.103246,0.054353,0.066289,0.045100,0.034454,0.035082,0.026971,0.011084
night,0.447566,0.177681,0.120468,0.045952,0.070826,0.034378,0.031950,0.034236,0.024046,0.012897


In [ ]:
#del glob_freq_tod
#gc.collect()

In [47]:
#glob_freq.keys()

## get difference from global freq target

In [67]:
%%time
# 154 ms

glob_diff_freq_dom  = pd.DataFrame(index = glob_freq_dom.index)
glob_diff_freq_dow  = pd.DataFrame(index = glob_freq_dow.index)
glob_diff_freq_hour = pd.DataFrame(index = glob_freq_hour.index)
glob_diff_freq_tod  = pd.DataFrame(index = glob_freq_tod.index)

for el in glob_freq.keys():
    glob_diff_freq_dom['diff_' + el]   = glob_freq[el] - glob_freq_dom[el]
    glob_diff_freq_dow['diff_' + el]   = glob_freq[el] - glob_freq_dow[el]
    glob_diff_freq_hour['diff_' + el]  = glob_freq[el] - glob_freq_hour[el]
    glob_diff_freq_tod['diff_' + el]   = glob_freq[el] - glob_freq_tod[el]
    #glob_diff_freq_dom['diff_' + el]  = glob_freq_dom[el].apply(diff_freq_main_screen)
    #glob_diff_freq_dow['diff_' + el]  = glob_freq_dow[el].apply()
    #glob_diff_freq_hour['diff_' + el] = glob_freq_hour[el].apply()
    #glob_diff_freq_tod['diff_' + el]  = glob_freq_tod[el].apply()
    
glob_diff_freq_dom.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dom.csv'))
glob_diff_freq_dow.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dow.csv'))
glob_diff_freq_hour.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_hour.csv'))
glob_diff_freq_tod.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_tod.csv'))

Wall time: 154 ms


In [69]:
#del glob_diff_freq_dom
#del glob_diff_freq_dow
#del glob_diff_freq_hour
#del glob_diff_freq_tod

#gc.collect()

In [20]:
def client_freq_blank(inp_series, feature):
    tmp = inp_series.value_counts()
    if feature in tmp.keys():
        return (tmp[feature]) / sum(tmp.values)
    else:
        return 0

#### client frequency of target for all dataset

In [21]:
def client_freq_main_screen(inp_series):
    return client_freq_blank(inp_series, 'main_screen')
    
def client_freq_statement(inp_series):
    return client_freq_blank(inp_series, 'statement')
    
def client_freq_credit_info(inp_series):
    return client_freq_blank(inp_series, 'credit_info')
    
def client_freq_own_transfer(inp_series):
    return client_freq_blank(inp_series, 'own_transfer')

def client_freq_mobile_recharge(inp_series):
    return client_freq_blank(inp_series, 'mobile_recharge')
          
def client_freq_phone_money_transfer(inp_series):
    return client_freq_blank(inp_series, 'phone_money_transfer')                             
     
def client_freq_card2card_transfer(inp_series):
    return client_freq_blank(inp_series, 'card2card_transfer')
       
def client_freq_chat(inp_series):
    return client_freq_blank(inp_series, 'chat')
                     
def client_freq_card_recharge(inp_series):
    return client_freq_blank(inp_series, 'card_recharge')
    
def client_freq_invest(inp_series):
    return client_freq_blank(inp_series, 'invest')

In [22]:
cli_for_agg = [client_freq_main_screen, client_freq_statement, 
               client_freq_credit_info, client_freq_own_transfer,
               client_freq_mobile_recharge, client_freq_phone_money_transfer, 
               client_freq_card2card_transfer, client_freq_chat, 
               client_freq_card_recharge, client_freq_invest
              ]

In [23]:
#%%timeit
#client_freq_main_screen(data.multi_class_target)

In [42]:
%%time
# 5min 46s

client_freq = data.groupby('client_pin').multi_class_target.agg(cli_for_agg)
client_freq.to_csv(os.path.join(DATA_OWN, 'client_freq.csv'))
client_freq.head(10)

Wall time: 6min 30s


,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,client_freq_chat,client_freq_card_recharge,client_freq_invest
client_pin,,,,,,,,,,
000033b6509acd1c8eb0d06ebd2e1de9,0.969466,0.015267,0.000000,0.0,0.015267,0.000000,0.000000,0.000000,0.000000,0.000000
0003653d4fa5e963ef379d65dd1d26ef,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0003a7947510434f8248b5b6a1eba0fa,0.636364,0.272727,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000
0005e7be2d89541ceea6b2f0bb4bc790,0.092308,0.000000,0.507692,0.0,0.000000,0.000000,0.000000,0.153846,0.246154,0.000000
00067f6a7575df6f067b3677b6ebfadc,0.564103,0.307692,0.076923,0.0,0.025641,0.000000,0.000000,0.025641,0.000000,0.000000
000967564d2bd81452b32fb6947421ee,0.692308,0.051282,0.051282,0.0,0.025641,0.025641,0.000000,0.153846,0.000000,0.000000
00096befff35b861d19c6de26db86fd6,0.689655,0.068966,0.068966,0.0,0.034483,0.068966,0.034483,0.000000,0.000000,0.034483
000a540f0ea28469451d4a7ad9e9436d,0.454545,0.151515,0.000000,0.0,0.181818,0.010101,0.191919,0.010101,0.000000,0.000000
000a90f770c38f235ff73e0f2aa7d197,0.542373,0.067797,0.000000,0.0,0.322034,0.050847,0.016949,0.000000,0.000000,0.000000


In [ ]:
#del client_freq
#gc.collect()

#### client frequency of target for day of week

In [25]:
%%time
# 32min 55s

client_freq_dow = data.groupby(['client_pin', 'dow']).multi_class_target.agg(cli_for_agg)
# after read_csv multilevel will disapper
client_freq_dow.to_csv(os.path.join(DATA_OWN, 'client_freq_dow.csv'))
client_freq_dow.head(10)

Wall time: 32min 55s


client_freq_main_screen  \
client_pin                       dow                            
000033b6509acd1c8eb0d06ebd2e1de9 0                   1.000000   
                                 1                   1.000000   
                                 2                   0.952381   
                                 3                   1.000000   
                                 4                   1.000000   
                                 5                   0.866667   
                                 6                   0.875000   
0003653d4fa5e963ef379d65dd1d26ef 0                   0.000000   
                                 4                   0.000000   
0003a7947510434f8248b5b6a1eba0fa 0                   0.666667   

                                      client_freq_statement  \
client_pin                       dow                          
000033b6509acd1c8eb0d06ebd2e1de9 0                 0.000000   
                                 1                 0.000000   
                                 2                 0.000000   
                                 3                 0.000000   
                                 4                 0.000000   
                                 5                 0.066667   
                                 6                 0.125000   
0003653d4fa5e963ef379d65dd1d26ef 0                 1.000000   
                                 4                 1.000000   
0003a7947510434f8248b5b6a1eba0fa 0                 0.333333   

                                      client_freq_credit_info  \
client_pin                       dow                            
000033b6509acd1c8eb0d06ebd2e1de9 0                        0.0   
                                 1                        0.0   
                                 2                        0.0   
                                 3                        0.0   
                                 4                        0.0   
                                 5                        0.0   
                                 6                        0.0   
0003653d4fa5e963ef379d65dd1d26ef 0                        0.0   
                                 4                        0.0   
0003a7947510434f8248b5b6a1eba0fa 0                        0.0   

                                      client_freq_own_transfer  \
client_pin                       dow                             
000033b6509acd1c8eb0d06ebd2e1de9 0                         0.0   
                                 1                         0.0   
                                 2                         0.0   
                                 3                         0.0   
                                 4                         0.0   
                                 5                         0.0   
                                 6                         0.0   
0003653d4fa5e963ef379d65dd1d26ef 0                         0.0   
                                 4                         0.0   
0003a7947510434f8248b5b6a1eba0fa 0                         0.0   

                                      client_freq_mobile_recharge  \
client_pin                       dow                                
000033b6509acd1c8eb0d06ebd2e1de9 0                       0.000000   
                                 1                       0.000000   
                                 2                       0.047619   
                                 3                       0.000000   
                                 4                       0.000000   
                                 5                       0.066667   
                                 6                       0.000000   
0003653d4fa5e963ef379d65dd1d26ef 0                       0.000000   
                                 4                       0.000000   
0003a7947510434f8248b5b6a1eba0fa 0                       0.000000   

                                      client_freq_phone_money_transfer  \
client_pin             

In [ ]:
#del client_freq_dow
#gc.collect()

#### client frequency of target for times of day

In [28]:
%%time
# 

client_freq_tod = data.groupby(['client_pin', 'tod']).multi_class_target.agg(cli_for_agg)
# after read_csv multilevel will disapper
client_freq_tod.to_csv(os.path.join(DATA_OWN, 'client_freq_tod.csv'))
client_freq_tod.head(10)

Wall time: 19min 37s


client_freq_main_screen  \
client_pin                       tod                                
000033b6509acd1c8eb0d06ebd2e1de9 day                     1.000000   
                                 evening                 0.955556   
                                 morning                 0.935484   
                                 night                   1.000000   
0003653d4fa5e963ef379d65dd1d26ef day                     0.000000   
0003a7947510434f8248b5b6a1eba0fa day                     0.500000   
                                 evening                 0.500000   
                                 morning                 1.000000   
                                 night                   0.666667   
0005e7be2d89541ceea6b2f0bb4bc790 day                     0.069767   

                                          client_freq_statement  \
client_pin                       tod                              
000033b6509acd1c8eb0d06ebd2e1de9 day                   0.000000   
                                 evening               0.044444   
                                 morning               0.000000   
                                 night                 0.000000   
0003653d4fa5e963ef379d65dd1d26ef day                   1.000000   
0003a7947510434f8248b5b6a1eba0fa day                   0.500000   
                                 evening               0.500000   
                                 morning               0.000000   
                                 night                 0.000000   
0005e7be2d89541ceea6b2f0bb4bc790 day                   0.000000   

                                          client_freq_credit_info  \
client_pin                       tod                                
000033b6509acd1c8eb0d06ebd2e1de9 day                     0.000000   
                                 evening                 0.000000   
                                 morning                 0.000000   
                                 night                   0.000000   
0003653d4fa5e963ef379d65dd1d26ef day                     0.000000   
0003a7947510434f8248b5b6a1eba0fa day                     0.000000   
                                 evening                 0.000000   
                                 morning                 0.000000   
                                 night                   0.000000   
0005e7be2d89541ceea6b2f0bb4bc790 day                     0.418605   

                                          client_freq_own_transfer  \
client_pin                       tod                                 
000033b6509acd1c8eb0d06ebd2e1de9 day                           0.0   
                                 evening                       0.0   
                                 morning                       0.0   
                                 night                         0.0   
0003653d4fa5e963ef379d65dd1d26ef day                           0.0   
0003a7947510434f8248b5b6a1eba0fa day                           0.0   
                                 evening                       0.0   
                                 morning                       0.0   
                                 night                         0.0   
0005e7be2d89541ceea6b2f0bb4bc790 day                           0.0   

                                          client_freq_mobile_recharge  \
client_pin                       tod                                    
000033b6509acd1c8eb0d06ebd2e1de9 day                         0.000000   
                                 evening                     0.000000   
                                 morning                     0.064516   
                                 night                       0.000000   
0003653d4fa5e963ef379d65dd1d26ef day                         0.000000   
0003a7947510434f8248b5b6a1eba0fa day                         0.000000   
                                 evening                     0.000000   
                                 morning                     0.000000   
        

In [ ]:
#del client_freq_tod
#gc.collect()

## get difference from client freq target

In [70]:
%%time
# 11.3 s

#client_diff_freq_dom  = pd.DataFrame(index = client_freq_dom.index)
client_diff_freq_dow  = pd.DataFrame(index = client_freq_dow.index)
#client_diff_freq_hour = pd.DataFrame(index = client_freq_hour.index)
client_diff_freq_tod  = pd.DataFrame(index = client_freq_tod.index)

for el in client_freq.keys():
#    client_diff_freq_dom['diff_' + el]   = client_freq[el] - client_freq_dom[el]
    client_diff_freq_dow['diff_' + el]   = client_freq[el] - client_freq_dow[el]
#    client_diff_freq_hour['diff_' + el]  = client_freq[el] - client_freq_hour[el]
    client_diff_freq_tod['diff_' + el]   = client_freq[el] - client_freq_tod[el]
    
#client_diff_freq_dom.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_dom.csv'))
client_diff_freq_dow.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_dow.csv'))
#client_diff_freq_hour.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_hour.csv'))
client_diff_freq_tod.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_tod.csv'))

Wall time: 11.3 s


In [71]:
#del client_diff_freq_dom
#del client_diff_freq_hour

#del client_diff_freq_dow
#del client_diff_freq_tod

#gc.collect()

# USED UNTIL THIS LINE

In [ ]:
data[data.client_pin == gg]

# time past after last known target for this user

In [ ]:
data.time_diff

In [ ]:
target_dict = {'main_screen': 0, 
               'statement': 1,
               'credit_info': 2,
               'own_transfer': 3,
               'mobile_recharge': 4,
               'phone_money_transfer': 5,
               'card2card_transfer': 6,
               'chat': 7,
               'card_recharge': 8,
               'invest': 9
}

def target_time_diff(inp_data):

    #print(inp_data.shape)
    #print(inp_data.client_pin)
    #ret_series = pd.Series([[0]]*inp_data.shape[0]
    #                       , index = inp_data.index)
    ret_series = pd.Series([[0]]*inp_data.shape[0]
                           , index = inp_data.index
                           , dtype = 'object'
                          )
    last_known = [0] * len(target_dict)
    #last_known = np.array([0] * len(target_dict))
    
    #print(ret_series.dtype)
    for idx, val in enumerate(inp_data.index):
        time  = inp_data['time_diff'][val]
        field = inp_data['multi_class_target'][val]
        
        last_known = [x+time for x in last_known]
        #ret_series.loc[val] = last_known
        ret_series.iloc[idx] = np.array(last_known, dtype = object)
        #update last known time:
        last_known[target_dict[field]] = 0
            
    #print(ret_series.shape)
    
    return ret_series

In [ ]:
#first = data[data.client_pin == gg].groupby('client_pin')
first = data.query('client_pin == @gg1 or client_pin == @gg2 or \
            client_pin == @gg3 or client_pin == @gg4 or client_pin == @gg5').groupby('client_pin')

In [ ]:
%%time
#.group['multi_class_target']
#tt = first.apply(target_time_diff)

tt = data.groupby('client_pin').progress_apply(target_time_diff)

In [ ]:
tt = tt.reset_index()

In [ ]:
tt.set_index('ind', inplace=True)

In [ ]:
tt.columns=['client_pin', 'last_known']

In [ ]:
with open(os.path.join(DATA_OWN, 'last_known.pkl'), 'wb') as fd_last_known:
    pickle.dump(tt, fd_last_known, protocol = 2)

In [ ]:
%%time
with open(os.path.join(DATA_OWN, 'last_known2.pkl'), 'wb') as fd_last_known:
    pickle.dump(tt, fd_last_known, protocol = 2)

In [ ]:
tt.index.name = ('client_pin', 'ind')

In [ ]:
%%time
#tt.to_csv(os.path.join(DATA_OWN, 'client_last_target.csv'))
dd = pd.read_csv(os.path.join(DATA_OWN, 'client_last_target.csv'))

In [ ]:
%%time
with open(os.path.join(DATA_OWN, 'last_known.pkl'), 'rb') as fd_last_known:
    tt2 = pickle.load(fd_last_known, protocol = 2)

In [ ]:
tt.index

In [ ]:
dd.columns = ['client_pin', 'ind', 'last_known']
dd.set_index('ind', inplace=True)

In [ ]:
dd.loc[4]

In [ ]:
data.loc[4]

In [ ]:
dd.head()

In [ ]:
for el in tqdm(target_dict.keys()):
    dd['last_known_' + el] = dd.last_known.apply(lambda x: x[target_dict[el]])

In [ ]:
target_dict.keys()

In [ ]:
dd

In [ ]:
dd.loc[0, 'last_known']

In [ ]:
from dask.distributed import Client
from dask import delayed

client = Client(n_workers=8)

In [ ]:
def prep_ds(inp_data):
    
    #global data
    #tmp_data = data.query('client_pin == @inp_client_pin')
    
    #return data.query('client_pin == @inp_client_pin').nunique()
    
    indexes = inp_data.index
    
    dict_freq = {'mobile_recharge': [0], 'statement': [0], 'phone_money_transfer': [0], 
             'chat': [0], 'invest': [0], 'main_screen': [0], 'own_transfer': [0], 
             'card_recharge': [0], 'credit_info': [0], 'card2card_transfer': [0]}
    
    tmp_value_counts = inp_data.multi_class_target.value_counts()
    sum_sess = inp_data.shape[0]
    for el in tmp_value_counts.keys():
        dict_freq[el] = [tmp_value_counts[el] / sum_sess]
    
    feature_dict = {}
    for idx, el in enumerate(indexes):
        feature_dict['last_before'] = 
        #'most_common',
        #'mc_last_24hours',
        #'mc_last_week',
        'days_since_last_days_since_last_mobile_recharge',
        'days_since_last_statement',
        'days_since_last_phone_money_transfer',
        'days_since_last_chat',
        'days_since_last_invest',
        'days_since_last_main_screen',
        'days_since_last_own_transfer',
        'days_since_last_card_recharge',
        'days_since_last_credit_info',
        'days_since_last_card2card_transfer'
    
    return inp_data.nunique()

In [ ]:
%%time
uniqq = []
for el in tqdm(data.client_pin.unique()[:100]):
    #uniqq.append(data.query('client_pin == @el').nunique())
    #ret = delayed(prep_ds)(el)
    ret = delayed(prep_ds)(data.query('client_pin == @el'))
    uniqq.append(ret)
    
ss = sum(uniqq)
answ = ss.compute()
#answ = sum(uniqq)
#ret.visalize()
print(answ)

In [ ]:
#ret.visualize()
client.close()

In [ ]:
data.client_pin.unique()[:5]